In [3]:
!pip install wandb ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.8 MB/s eta 0:00:0000:01


In [4]:
import ultralytics
from ultralytics import YOLO
import wandb
from datetime import datetime

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
# Login to Google Drive
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
wandb.login()

In [ ]:
%%yolo settings wandb=True

In [ ]:
DATA_DIR = ""

In [ ]:
with wandb.init(project="contrail-segmentation", entity="shrenikborad") as run:
    config = wandb.config
    config.data_dir = DATA_DIR

    model = YOLO("yolo11m-seg.pt")

    model.train(
        data=f"{config.data_dir}/data.yaml",
        epochs=100,
        imgsz=640,
        name="contrail-segmentation-run",
        exist_ok=True,
        batch=-1,
        save_period=5,
        project="/content/drive/MyDrive/ColabNotebooks/pless/contrail-seg/runs",

    )